# 브랜드별 매장 데이터 전처리

- 목표 
    - 브랜드별 크롤링 결과를 묶어서 데이터 분석할 수 있는 데이터프레임을 생성한다. 

- 주요 내용
    - 컬럼명 획일화 (예) "지점명", "매장명" → "지점명"으로
    - 주소 행정구역명 획일화 (예) "서울시","서울특별시","서울" → "서울특별시"로

- 최종 데이터프레임 
    - 객체명 : `burger_df`
    - 크기 : (2306, 6)
    - 컬럼명 : '지점명', '주소', '브랜드', '주소1', '주소2', '주소3'
    - 파일명 : *burger_df.csv* 또는 *burger_df.xlsx*로 저장
    - 구성  
    
|index|브랜드|주소|지점명|주소1(시,도)|주소2(시,구,군)|주소3(나머지)|
|---|---|---|---|---|---|---|
|0|버거킹| 서울특별시 강남구 선릉로 429	| 선릉역점	| 서울특별시| 강남구 | 선릉로 429|
|...|...|...|...|...|...|...|...|

- 특이사항
    - 롯데리아의 경우 크롤링한 결과 자체가 최종 데이터프레임과 동일한 구성으로 되어있기 때문에 나머지 브랜드를 처리한 후에 `pandas.concat` 이용하여 추가
   


In [50]:
import pandas as pd
import os

In [29]:
# 각 브랜드별 DataFrame 생성
burger_king_df = pd.read_excel('./BurgerKing/BurgerKing_all_store.xlsx')
kfc_df = pd.read_excel('./KFC/KFC_all_store.xlsx')
mcdonalds_df = pd.read_excel('./Mcdonalds/mcdonalds_all_store.xlsx')
lotteria_df = pd.read_excel('./Lotteria/lotteria_all_store.xlsx')

In [30]:
burger_king_df.head()

,지점명,주소
0,선릉역점,서울특별시 강남구 선릉로 429
1,대치역점,서울특별시 강남구 남부순환로 2936
2,차병원사거리점,서울특별시 강남구 봉은사로 179
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층
4,청담점,서울특별시 강남구 선릉로 812


In [31]:
kfc_df.head()

,매장명,매장 주소
0,원주단계DT,강원 원주시 북원로 2266 (단계동) KFC원주단계DT
1,춘천이마트,강원 춘천시 경춘로 2341 (온의동) 이마트 춘천점 1층 푸드코트 內
2,춘천석사,강원 춘천시 영서로 2027 (석사동)
3,행신역,경기 고양시 덕양구 충장로 8 (행신동)
4,화정,경기 고양시 덕양구 화신로272번길 57 (화정동)


In [32]:
mcdonalds_df.head()

,매장명,매장 주소
0,고양 삼송 DT,경기 고양시 덕양구 고양대로 1948
1,서울시청,서울 중구 남대문로9길 51 효덕빌딩
2,한신,서울 노원구 한글비석로 57 (하계동)
3,영천DT,경북 영천시 호국로 141
4,대구동호DT,대구 동구 안심로 403


In [33]:
lotteria_df.head()

,지점명,주소,주소1,주소2,주소3
0,가평휴게소(상),경기 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544
1,수원파장,경기 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91
2,대구성당D/T,대구 달서구 구마로 209,대구광역시,달서구,구마로 209
3,아산장재,서울 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47
4,김포마산,경기 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5


In [34]:
burger_king_df.shape, kfc_df.shape, mcdonalds_df.shape, lotteria_df.shape

((423, 2), (187, 2), (405, 2), (1304, 5))

In [35]:
# 각 DataFrame에 브랜드 열 생성
burger_king_df['브랜드'] = '버거킹'
kfc_df['브랜드'] = 'KFC'
mcdonalds_df['브랜드'] = '맥도날드'
lotteria_df['브랜드'] = '롯데리아'

In [36]:
# 중복 데이터 확인
print( burger_king_df['지점명'].value_counts().sort_values(ascending=False).head(1) )
print()
print( kfc_df['매장명'].value_counts().sort_values(ascending=False).head(1) )
print()
print( mcdonalds_df['매장명'].value_counts().sort_values(ascending=False).head(1) )
print()
print( lotteria_df['지점명'].value_counts().sort_values(ascending=False).head(1) )

남양주진접FS점    2
Name: 지점명, dtype: int64

불광역    1
Name: 매장명, dtype: int64

대구복현DT점    1
Name: 매장명, dtype: int64

안동옥동    1
Name: 지점명, dtype: int64


In [37]:
# 중복 제거
burger_king_df.drop_duplicates('지점명', keep='first', inplace=True)
kfc_df.drop_duplicates('매장명', keep='first', inplace=True)
mcdonalds_df.drop_duplicates('매장명', keep='first', inplace=True)
lotteria_df.drop_duplicates('지점명', keep='first', inplace=True)

burger_king_df.shape, kfc_df.shape, mcdonalds_df.shape, lotteria_df.shape

((410, 3), (187, 3), (405, 3), (1304, 6))

In [38]:
# 빈 DataFrame 생성 후 각 브랜드별 DataFrame 추가
burger_df = pd.DataFrame()

burger_df = pd.concat([burger_df, burger_king_df])
burger_df = pd.concat([burger_df, kfc_df])
burger_df = pd.concat([burger_df, mcdonalds_df])
# burger_df = pd.concat([burger_df, lotteria_df])

# DataFrame 인덱스 재설정
burger_df.reset_index(drop=True, inplace=True)

In [39]:
burger_df.head()

,지점명,주소,브랜드,매장명,매장 주소
0,선릉역점,서울특별시 강남구 선릉로 429,버거킹,NaN,NaN
1,대치역점,서울특별시 강남구 남부순환로 2936,버거킹,NaN,NaN
2,차병원사거리점,서울특별시 강남구 봉은사로 179,버거킹,NaN,NaN
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,버거킹,NaN,NaN
4,청담점,서울특별시 강남구 선릉로 812,버거킹,NaN,NaN


In [40]:
# 매장명, 매장 주소를 지점명, 주소로 통일
burger_df['지점명'].fillna(burger_df['매장명'], inplace=True)
burger_df['주소'].fillna(burger_df['매장 주소'], inplace=True)
print(burger_df.isnull().sum())
burger_df.head()

지점명        0
주소         0
브랜드        0
매장명      410
매장 주소    410
dtype: int64


,지점명,주소,브랜드,매장명,매장 주소
0,선릉역점,서울특별시 강남구 선릉로 429,버거킹,NaN,NaN
1,대치역점,서울특별시 강남구 남부순환로 2936,버거킹,NaN,NaN
2,차병원사거리점,서울특별시 강남구 봉은사로 179,버거킹,NaN,NaN
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,버거킹,NaN,NaN
4,청담점,서울특별시 강남구 선릉로 812,버거킹,NaN,NaN


In [41]:
# 매장명, 매장 주소 열 삭제
burger_df.drop(['매장명', '매장 주소'], axis=1, inplace=True)
burger_df

,지점명,주소,브랜드
0,선릉역점,서울특별시 강남구 선릉로 429,버거킹
1,대치역점,서울특별시 강남구 남부순환로 2936,버거킹
2,차병원사거리점,서울특별시 강남구 봉은사로 179,버거킹
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,버거킹
4,청담점,서울특별시 강남구 선릉로 812,버거킹
...,...,...,...
997,대전카이스트점,대전 유성구 대덕대로 535,맥도날드
998,김천평화DT점,경북 김천시 자산로 199,맥도날드
999,대구태전 DT,대구 북구 칠곡중앙대로 303,맥도날드
1000,강남 2호점,서울 강남구 테헤란로 107 메디타워2층,맥도날드


In [42]:
# 주소를 시군구별로 나눠서 열 추가 후 재정렬
burger_df[['주소1', '주소2', '주소3']] = pd.DataFrame(burger_df['주소'].str.strip().str.split(' ', 2).tolist())
burger_df.reindex(columns=['브랜드', '지점명', '주소', '주소1', '주소2', '주소3'])

,브랜드,지점명,주소,주소1,주소2,주소3
0,버거킹,선릉역점,서울특별시 강남구 선릉로 429,서울특별시,강남구,선릉로 429
1,버거킹,대치역점,서울특별시 강남구 남부순환로 2936,서울특별시,강남구,남부순환로 2936
2,버거킹,차병원사거리점,서울특별시 강남구 봉은사로 179,서울특별시,강남구,봉은사로 179
3,버거킹,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,서울특별시,강남구,논현로 172 (도곡동 410-10) 1층
4,버거킹,청담점,서울특별시 강남구 선릉로 812,서울특별시,강남구,선릉로 812
...,...,...,...,...,...,...
997,맥도날드,대전카이스트점,대전 유성구 대덕대로 535,대전,유성구,대덕대로 535
998,맥도날드,김천평화DT점,경북 김천시 자산로 199,경북,김천시,자산로 199
999,맥도날드,대구태전 DT,대구 북구 칠곡중앙대로 303,대구,북구,칠곡중앙대로 303
1000,맥도날드,강남 2호점,서울 강남구 테헤란로 107 메디타워2층,서울,강남구,테헤란로 107 메디타워2층


In [43]:
# 시도 값 확인
burger_df['주소1'].unique()

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도',
       '제주특별자치도', '강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울',
       '세종특별자치시', '울산', '인천', '전남', '전북', '충남', '충북', '서울시'], dtype=object)

In [44]:
# 줄여쓴 구역을 풀어서 다시 입력
burger_df['주소1'] = burger_df['주소1'].replace({'서울시':'서울특별시',
                                                '서울':'서울특별시',
                                                '부산':'부산광역시',
                                                '대구':'대구광역시',
                                                '인천':'인천광역시',
                                                '광주':'광주광역시',
                                                '대전':'대전광역시',
                                                '울산':'울산광역시',
                                                '세종':'세종특별자치시',
                                                '경기':'경기도',
                                                '강원':'강원도',
                                                '충북':'충청북도',
                                                '충남':'충청남도',
                                                '전북':'전라북도',
                                                '전남':'전라남도',
                                                '경북':'경상북도',
                                                '경남':'경상남도',
                                                '제주':'제주특별자치도'})

burger_df

,지점명,주소,브랜드,주소1,주소2,주소3
0,선릉역점,서울특별시 강남구 선릉로 429,버거킹,서울특별시,강남구,선릉로 429
1,대치역점,서울특별시 강남구 남부순환로 2936,버거킹,서울특별시,강남구,남부순환로 2936
2,차병원사거리점,서울특별시 강남구 봉은사로 179,버거킹,서울특별시,강남구,봉은사로 179
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,버거킹,서울특별시,강남구,논현로 172 (도곡동 410-10) 1층
4,청담점,서울특별시 강남구 선릉로 812,버거킹,서울특별시,강남구,선릉로 812
...,...,...,...,...,...,...
997,대전카이스트점,대전 유성구 대덕대로 535,맥도날드,대전광역시,유성구,대덕대로 535
998,김천평화DT점,경북 김천시 자산로 199,맥도날드,경상북도,김천시,자산로 199
999,대구태전 DT,대구 북구 칠곡중앙대로 303,맥도날드,대구광역시,북구,칠곡중앙대로 303
1000,강남 2호점,서울 강남구 테헤란로 107 메디타워2층,맥도날드,서울특별시,강남구,테헤란로 107 메디타워2층


In [45]:
# 주소1, 주소2, 주소3 을 합쳐서 주소를 다시 입력
burger_df['주소'] = burger_df[ ['주소1', '주소2', '주소3'] ].apply(' '.join, axis=1)
burger_df

,지점명,주소,브랜드,주소1,주소2,주소3
0,선릉역점,서울특별시 강남구 선릉로 429,버거킹,서울특별시,강남구,선릉로 429
1,대치역점,서울특별시 강남구 남부순환로 2936,버거킹,서울특별시,강남구,남부순환로 2936
2,차병원사거리점,서울특별시 강남구 봉은사로 179,버거킹,서울특별시,강남구,봉은사로 179
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,버거킹,서울특별시,강남구,논현로 172 (도곡동 410-10) 1층
4,청담점,서울특별시 강남구 선릉로 812,버거킹,서울특별시,강남구,선릉로 812
...,...,...,...,...,...,...
997,대전카이스트점,대전광역시 유성구 대덕대로 535,맥도날드,대전광역시,유성구,대덕대로 535
998,김천평화DT점,경상북도 김천시 자산로 199,맥도날드,경상북도,김천시,자산로 199
999,대구태전 DT,대구광역시 북구 칠곡중앙대로 303,맥도날드,대구광역시,북구,칠곡중앙대로 303
1000,강남 2호점,서울특별시 강남구 테헤란로 107 메디타워2층,맥도날드,서울특별시,강남구,테헤란로 107 메디타워2층


In [46]:
lotteria_df

,지점명,주소,주소1,주소2,주소3,브랜드
0,가평휴게소(상),경기 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544,롯데리아
1,수원파장,경기 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91,롯데리아
2,대구성당D/T,대구 달서구 구마로 209,대구광역시,달서구,구마로 209,롯데리아
3,아산장재,서울 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47,롯데리아
4,김포마산,경기 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5,롯데리아
...,...,...,...,...,...,...
1299,여의도선착장,서울 영등포구 여의동로 290,서울특별시,영등포구,여의동로 290,롯데리아
1300,청주하나로,충북 청주시 상당구 무농정로 32 청주하나로클럽內,충청북도,청주시,상당구 무농정로 32 청주하나로클럽內,롯데리아
1301,전주효자서부,전북 전주시 완산구 호암로 65,전라북도,전주시,완산구 호암로 65,롯데리아
1302,시흥정왕역,경기 시흥시 마유로418번 길10,경기도,시흥시,마유로418번 길10,롯데리아


In [47]:
# 롯데리아 데이터 추가
burger_df = pd.concat([burger_df, lotteria_df])
burger_df.reset_index(drop=True, inplace=True)
burger_df

,지점명,주소,브랜드,주소1,주소2,주소3
0,선릉역점,서울특별시 강남구 선릉로 429,버거킹,서울특별시,강남구,선릉로 429
1,대치역점,서울특별시 강남구 남부순환로 2936,버거킹,서울특별시,강남구,남부순환로 2936
2,차병원사거리점,서울특별시 강남구 봉은사로 179,버거킹,서울특별시,강남구,봉은사로 179
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,버거킹,서울특별시,강남구,논현로 172 (도곡동 410-10) 1층
4,청담점,서울특별시 강남구 선릉로 812,버거킹,서울특별시,강남구,선릉로 812
...,...,...,...,...,...,...
2301,여의도선착장,서울 영등포구 여의동로 290,롯데리아,서울특별시,영등포구,여의동로 290
2302,청주하나로,충북 청주시 상당구 무농정로 32 청주하나로클럽內,롯데리아,충청북도,청주시,상당구 무농정로 32 청주하나로클럽內
2303,전주효자서부,전북 전주시 완산구 호암로 65,롯데리아,전라북도,전주시,완산구 호암로 65
2304,시흥정왕역,경기 시흥시 마유로418번 길10,롯데리아,경기도,시흥시,마유로418번 길10


In [51]:
# csv, xlsx 파일로 저장
burger_df.to_csv("burger_df.csv", index=False)
burger_df.to_excel("burger_df.xlsx", index=False)

os.listdir()

['burger_index_preprocessing.ipynb',
 'Mcdonalds',
 '.DS_Store',
 'BurgerKing',
 'KFC',
 'README.md',
 'burger_df.csv',
 'burger_df.xlsx',
 'burger_index.ipynb',
 '.ipynb_checkpoints',
 'stat_100701.xls',
 '.git',
 'Lotteria']

In [ ]:
# 주소 전처리 종료